In [ ]:
#Citations
#https://www.kaggle.com/c/home-credit-default-risk/discussion/64821
#https://www.kaggle.com/shanth84/home-credit-bureau-data-feature-engineering?scriptVersionId=3798236
#The techniques in our code was learn't from the above notebook

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import gc

In [2]:
ls

 data/                              lgbm_sub.csv
 Data_Exploration_FullSet.ipynb     LightGBM.ipynb
 Data_Exploration.ipynb             Model_Comparison.ipynb
 Feature_Eng.ipynb                 'Model Performance.ipynb'
 GridsearchWIP.ipynb                test_feat_eng.csv
 LGBM_hyperparameterization.ipynb   train_feat_eng.csv


In [3]:
# load train and test sets
train = pd.read_csv('data/application_train.csv')
test = pd.read_csv('data/application_test.csv')

In [4]:
credit_card_balance = pd.read_csv('data/credit_card_balance.csv')
credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [5]:
cred_limit = credit_card_balance.groupby('SK_ID_CURR', as_index=False)['AMT_CREDIT_LIMIT_ACTUAL'].mean()
cred_limit.head()

,SK_ID_CURR,AMT_CREDIT_LIMIT_ACTUAL
0,100006,270000.000000
1,100011,164189.189189
2,100013,131718.750000
3,100021,675000.000000
4,100023,135000.000000


In [6]:
train = train.merge(cred_limit, on = 'SK_ID_CURR', how = 'left')
test = test.merge(cred_limit, on = 'SK_ID_CURR', how = 'left')

train['AMT_CREDIT_LIMIT_ACTUAL'] = train['AMT_CREDIT_LIMIT_ACTUAL'].fillna(0)
train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_CREDIT_LIMIT_ACTUAL
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,270000.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
prev_app = pd.read_csv('data/previous_application.csv')
prev_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
prev_app['SK_ID_CURR'].duplicated().any()

True

In [9]:
bureau = pd.read_csv('data/bureau.csv')
bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN


In [10]:
bureau['SK_ID_CURR'].duplicated().any()

True

In [11]:
bureau['CREDIT_ACTIVE'].unique()

array(['Closed', 'Active', 'Sold', 'Bad debt'], dtype=object)

In [12]:
B = bureau.copy()
grp = B.groupby(by = ['SK_ID_CURR'])['CREDIT_DAY_OVERDUE'].sum().reset_index().rename(index=str, columns={'CREDIT_DAY_OVERDUE': 'OVERDUE_DAYS_SUM'})
B = B.merge(grp, on = ['SK_ID_CURR'], how = 'left')
train = train.merge(grp, on = ['SK_ID_CURR'], how = 'left')
test = test.merge(grp, on = ['SK_ID_CURR'], how = 'left')
train
gc.collect()
print(B.shape)

(1716428, 18)


In [13]:
B

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,OVERDUE_DAYS_SUM
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN,0
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN,0
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN,0
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN,0
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN,0
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN,0
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN,0
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN,0


In [14]:
#lets look at how much the person spends
#specifally looks at people who take a lot of money out from ATMs.............
CC = credit_card_balance.copy()
grp = CC.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_ATM_CURRENT'].sum().reset_index().rename(index=str, columns={'AMT_DRAWINGS_ATM_CURRENT': 'TOTAL_ATM_WITHDRAW'})
CC = CC.merge(grp, on = ['SK_ID_CURR'], how = 'left')
train = train.merge(grp, on = ['SK_ID_CURR'], how = 'left')
test = test.merge(grp, on = ['SK_ID_CURR'], how = 'left')
train
gc.collect()
print(CC.shape)

(3840312, 24)


In [15]:
CC

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,TOTAL_ATM_WITHDRAW
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.0,1,0.0,1.0,35.0,Active,0,0,67500.0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0,293850.0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0,506250.0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0,50850.0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0,763650.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,1036507,328243,-9,0.000,45000,NaN,0.0,NaN,NaN,0.000,...,0.000,NaN,0,NaN,NaN,0.0,Active,0,0,0.0
3840308,1714892,347207,-9,0.000,45000,0.0,0.0,0.0,0.0,0.000,...,0.000,0.0,0,0.0,0.0,23.0,Active,0,0,189000.0
3840309,1302323,215757,-9,275784.975,585000,270000.0,270000.0,0.0,0.0,2250.000,...,273093.975,2.0,2,0.0,0.0,18.0,Active,0,0,6606000.0
3840310,1624872,430337,-10,0.000,450000,NaN,0.0,NaN,NaN,0.000,...,0.000,NaN,0,NaN,NaN,0.0,Active,0,0,0.0


In [16]:
test[test['TOTAL_ATM_WITHDRAW'].isna()].count()

SK_ID_CURR                    32091
NAME_CONTRACT_TYPE            32091
CODE_GENDER                   32091
FLAG_OWN_CAR                  32091
FLAG_OWN_REALTY               32091
                              ...  
AMT_REQ_CREDIT_BUREAU_QRT     27862
AMT_REQ_CREDIT_BUREAU_YEAR    27862
AMT_CREDIT_LIMIT_ACTUAL           0
OVERDUE_DAYS_SUM              27609
TOTAL_ATM_WITHDRAW                0
Length: 124, dtype: int64

In [17]:
test[~test['TOTAL_ATM_WITHDRAW'].isna()].head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_CREDIT_LIMIT_ACTUAL,OVERDUE_DAYS_SUM,TOTAL_ATM_WITHDRAW
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0.0,0.0,0.0,0.0,1.0,4.0,131718.75,0.0,571500.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0.0,0.0,0.0,0.0,0.0,3.0,225000.00,0.0,27000.0
5,100042,Cash loans,F,Y,Y,0,270000.0,959688.0,34600.5,810000.0,...,0,0.0,0.0,0.0,0.0,1.0,2.0,78750.00,0.0,265500.0
8,100066,Cash loans,F,N,Y,0,315000.0,364896.0,28957.5,315000.0,...,0,0.0,0.0,0.0,0.0,0.0,5.0,72600.00,0.0,0.0
9,100067,Cash loans,F,Y,Y,1,162000.0,45000.0,5337.0,45000.0,...,0,0.0,0.0,0.0,0.0,0.0,2.0,45000.00,0.0,66600.0


In [18]:
train['AMT_CREDIT_TO_ANNUITY_RATIO'] = train['AMT_CREDIT'] / train['AMT_ANNUITY']
test['AMT_CREDIT_TO_ANNUITY_RATIO'] = test['AMT_CREDIT'] / test['AMT_ANNUITY']

In [19]:
goods_cols = [col for col in train.columns if 'GOODS' in col]
goods_cols

['AMT_GOODS_PRICE']

In [20]:
train['AMT_CREDIT_GOODS_RATIO'] = train['AMT_CREDIT'] / train['AMT_GOODS_PRICE']
test['AMT_CREDIT_GOODS_RATIO'] = test['AMT_CREDIT'] / test['AMT_GOODS_PRICE']

In [21]:
AMT_cols = [col for col in train.columns if 'AMT' in col]
AMT_cols

['AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_QRT',
 'AMT_REQ_CREDIT_BUREAU_YEAR',
 'AMT_CREDIT_LIMIT_ACTUAL',
 'AMT_CREDIT_TO_ANNUITY_RATIO',
 'AMT_CREDIT_GOODS_RATIO']

In [22]:
train[AMT_cols]

,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_CREDIT_LIMIT_ACTUAL,AMT_CREDIT_TO_ANNUITY_RATIO,AMT_CREDIT_GOODS_RATIO
0,202500.0,406597.5,24700.5,351000.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,16.461104,1.158397
1,270000.0,1293502.5,35698.5,1129500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.234085,1.145199
2,67500.0,135000.0,6750.0,135000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.000000,1.000000
3,135000.0,312682.5,29686.5,297000.0,NaN,NaN,NaN,NaN,NaN,NaN,270000.0,10.532818,1.052803
4,121500.0,513000.0,21865.5,513000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.461618,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,157500.0,254700.0,27558.0,225000.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,9.242325,1.132000
307507,72000.0,269550.0,12001.5,225000.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,22.459693,1.198000
307508,153000.0,677664.0,29979.0,585000.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,22.604623,1.158400
307509,171000.0,370107.0,20205.0,319500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.317595,1.158394


In [23]:
train['AMT_CREDIT_INCOME_RATIO'] = train['AMT_CREDIT'] / train['AMT_INCOME_TOTAL']
test['AMT_CREDIT_INCOME_RATIO'] = test['AMT_CREDIT'] / test['AMT_INCOME_TOTAL']

In [24]:
train[~train['OVERDUE_DAYS_SUM'].isna()].head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_CREDIT_LIMIT_ACTUAL,OVERDUE_DAYS_SUM,TOTAL_ATM_WITHDRAW,AMT_CREDIT_TO_ANNUITY_RATIO,AMT_CREDIT_GOODS_RATIO,AMT_CREDIT_INCOME_RATIO
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,1.0,0.0,0.0,NaN,16.461104,1.158397,2.007889
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,36.234085,1.145199,4.790750
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,20.000000,1.000000,2.000000
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,23.461618,1.000000,4.222222
5,100008,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,...,0.0,0.0,1.0,1.0,0.0,0.0,NaN,17.824857,1.079198,4.954500


In [25]:
# get number of past loans per customer 
B = bureau.copy()
grp = B[['SK_ID_CURR', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT'].count().reset_index().rename(index=str, columns={'DAYS_CREDIT': 'NUM_PREV_LOANS'})
B = B.merge(grp, on = ['SK_ID_CURR'], how = 'left')

In [26]:
# get percentage of active loans: more loans can potential mean a default

B['CREDIT_ACTIVE_BINARY'] = B['CREDIT_ACTIVE']
B['CREDIT_ACTIVE_BINARY'] = B.apply(lambda x: x.CREDIT_ACTIVE == 'Closed', axis = 1)

# Calculate mean number of loans that are ACTIVE per CUSTOMER 
grp = B.groupby(by = ['SK_ID_CURR'])['CREDIT_ACTIVE_BINARY'].mean().reset_index().rename(index=str, columns={'CREDIT_ACTIVE_BINARY': 'ACTIVE_LOANS_PERCENTAGE'})
B = B.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del B['CREDIT_ACTIVE_BINARY']
gc.collect()

0

In [27]:
B['AMT_CREDIT_SUM_DEBT'] = B['AMT_CREDIT_SUM_DEBT'].fillna(0)
B['AMT_CREDIT_SUM'] = B['AMT_CREDIT_SUM'].fillna(0)

grp1 = B[['SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_DEBT': 'TOTAL_CUSTOMER_DEBT'})
grp2 = B[['SK_ID_CURR', 'AMT_CREDIT_SUM']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM': 'TOTAL_CUSTOMER_CREDIT'})

B = B.merge(grp1, on = ['SK_ID_CURR'], how = 'left')
B = B.merge(grp2, on = ['SK_ID_CURR'], how = 'left')
del grp1, grp2
gc.collect()

B['DEBT_CREDIT_RATIO'] = B['TOTAL_CUSTOMER_DEBT']/B['TOTAL_CUSTOMER_CREDIT']

del B['TOTAL_CUSTOMER_DEBT'], B['TOTAL_CUSTOMER_CREDIT']
gc.collect()

0

In [28]:
# merge new columns
B = B[["SK_ID_CURR","NUM_PREV_LOANS", "ACTIVE_LOANS_PERCENTAGE", "DEBT_CREDIT_RATIO"]]
B = B.drop_duplicates()
train = train.merge(B, on = 'SK_ID_CURR', how = 'left')
test = test.merge(B, on = 'SK_ID_CURR', how = 'left')

In [29]:
train[~train['ACTIVE_LOANS_PERCENTAGE'].isna()]

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_CREDIT_LIMIT_ACTUAL,OVERDUE_DAYS_SUM,TOTAL_ATM_WITHDRAW,AMT_CREDIT_TO_ANNUITY_RATIO,AMT_CREDIT_GOODS_RATIO,AMT_CREDIT_INCOME_RATIO,NUM_PREV_LOANS,ACTIVE_LOANS_PERCENTAGE,DEBT_CREDIT_RATIO
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,1.0,0.0,0.0,NaN,16.461104,1.158397,2.007889,8.0,0.750000,0.284122
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,NaN,36.234085,1.145199,4.790750,4.0,0.750000,0.000000
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,NaN,20.000000,1.000000,2.000000,2.0,1.000000,0.000000
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,NaN,23.461618,1.000000,4.222222,1.0,1.000000,0.000000
5,100008,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,...,1.0,0.0,0.0,NaN,17.824857,1.079198,4.954500,3.0,0.666667,0.512454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307503,456247,0,Cash loans,F,N,Y,0,112500.0,345510.0,17770.5,...,2.0,144000.0,0.0,202950.0,19.442897,1.396000,3.071200,11.0,0.727273,0.422988
307505,456249,0,Cash loans,F,N,Y,0,112500.0,225000.0,22050.0,...,0.0,0.0,0.0,NaN,10.204082,1.000000,2.000000,13.0,0.846154,0.044147
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,1.0,0.0,0.0,NaN,22.604623,1.158400,4.429176,4.0,0.500000,0.453493
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0.0,0.0,0.0,NaN,18.317595,1.158394,2.164368,1.0,1.000000,0.000000


In [30]:
train['SK_ID_CURR'].duplicated().any()

False

In [31]:
print(train.shape)
print(test.shape)

(307511, 131)
(48744, 130)


In [32]:
# output engineered CSVs 
train.to_csv("train_feat_eng.csv")
test.to_csv("test_feat_eng.csv")